In [1]:
import torch
from tqdm import tqdm
from torch.distributions import Normal

In [2]:
class MetaLearner(torch.nn.Module):
    def __init__(self, num_hidden=256):
        super(MetaLearner, self).__init__()
        
        self.rnn = torch.nn.LSTM(1, num_hidden)
        self.linear_mu = torch.nn.Linear(num_hidden, 1)
        self.linear_logstd = torch.nn.Linear(num_hidden, 1)

    def forward(self, inputs):
        """
        Args:
            inputs (Tensor): PyTorch tensor of size (seq_length, batch_size, 1)
        
        Returns:
            Normal object: Normal distribution object with dimensions (batch_size, 1)
        """
        hidden, _ = self.rnn(inputs)
        hidden = hidden[-1]
        
        mu = self.linear_mu(hidden)
        std = torch.exp(self.linear_logstd(hidden))
        
        return Normal(mu, std)

In [3]:
# parameters of data-generating distribution
prior_mean = 10; prior_std = 3; obs_std = 2; seq_length = 7

# parameters for training
iterations = 10000; batch_size = 32; num_runs = 30

In [4]:
losses = torch.zeros(num_runs, iterations + 1)

for run in range(num_runs):
    network = MetaLearner()
    optimizer = torch.optim.SGD(network.parameters(), lr=0.001)

    for t in tqdm(range(iterations + 1)):
        if not (t % 1000):
            torch.save(network, 'trained_models/iter_' + str(t) + '.pth')
            
        # sample data
        mu = Normal(prior_mean * torch.ones(1), prior_std).sample((batch_size,))
        x = Normal(mu, obs_std).sample((seq_length,))

        # forward pass
        predictive_posterior = network(x[:seq_length-1])

        # backward pass and update
        loss = -predictive_posterior.log_prob(x[-1]).mean()
        losses[run, t] = loss.item()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(), 40.0)
        optimizer.step()
torch.save(losses, 'losses.pth')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10001/10001 [03:48<00:00, 43.76it/s]
